<a href="https://colab.research.google.com/github/eth0-02/Astro-Theme-Creek/blob/master/Spatial_join_sub_with_basin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# ===== ONE-CELL COLAB: Basin name (original case) + SUB-basin HYBAS_ID =====
# Produces GPKG + CSV with BASIN_NAME, SUB_HYBAS_ID_STR, BASIN_HYBAS_ID (e.g., "Athi 1050008110")

# Deps
!apt-get -qq update
!apt-get -qq install -y libspatialindex-dev >/dev/null
!pip -q install geopandas shapely pyproj fiona rtree >/dev/null

import geopandas as gpd
import pandas as pd
import numpy as np
import re
from pathlib import Path
from google.colab import files

print("📤 Upload your TWO GeoPackages (basins & sub-basins)…")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No files uploaded.")
paths = [f"/content/{n}" for n in uploaded.keys()]
print("Uploaded:", paths)

def guess_role(p):  # filename heuristic
    return "sub" if "sub" in Path(p).name.lower() else "basin"

try:
    basin_file = next(p for p in paths if guess_role(p) == "basin")
    sub_file   = next(p for p in paths if guess_role(p) == "sub")
except StopIteration:
    if len(paths) == 2:
        basin_file, sub_file = paths[0], paths[1]
    else:
        raise SystemExit("Could not identify basins vs sub-basins; ensure one filename contains 'sub' or upload exactly 2 files.")

print(f"🗂️ Basins file:     {basin_file}")
print(f"🗂️ Sub-basins file: {sub_file}")

# Layers
import fiona
basin_layer = fiona.listlayers(basin_file)[0]
sub_layer   = fiona.listlayers(sub_file)[0]
print(f"📚 Basin layer: {basin_layer}")
print(f"📚 Sub layer:   {sub_layer}")

# Load
basins = gpd.read_file(basin_file, layer=basin_layer)
subs   = gpd.read_file(sub_file,   layer=sub_layer)
print(f"✅ Basins: {len(basins)} features, CRS={basins.crs}")
print(f"✅ Subs:   {len(subs)} features, CRS={subs.crs}")

# Reproject to meters (prefer basins CRS if projected; else EPSG:21037)
def ensure_projected(gdf, target="EPSG:21037"):
    if gdf.crs is None:
        print(f"• CRS missing → assigning {target}")
        return gdf.set_crs(target, allow_override=True)
    if gdf.crs.is_geographic:
        print(f"• Geographic CRS → reprojecting to {target}")
        return gdf.to_crs(target)
    return gdf

target_crs = basins.crs if (basins.crs and not basins.crs.is_geographic) else "EPSG:21037"
basins = ensure_projected(basins, target_crs)
subs   = ensure_projected(subs,   target_crs)
if basins.crs != subs.crs:
    subs = subs.to_crs(basins.crs)

# --- Pick fields (prefer BNAME for basin name; HYBAS_ID for ids) ---
def pick_hybas(df):
    for c in ["HYBAS_ID","Hybas_ID","hybas_id","HYBASID","HYBAS_LEV","HYBAS_LEV6","HYBAS","hybas"]:
        if c in df.columns: return c
    cand = [c for c in df.columns if re.search(r"hybas", c, re.I)]
    if cand: return cand[0]
    raise ValueError("No HYBAS field found.")

def pick_basin_name(df):
    if "BNAME" in df.columns: return "BNAME"           # your file has this
    for c in ["basin_name","BASIN_NAME","Basin_name","BASIN","Basin","NAME","Name"]:
        if c in df.columns: return c
    cand = [c for c in df.columns if df[c].dtype == object and re.search(r"(name|basin)", c, re.I)]
    return cand[0] if cand else None

BASIN_NAME_FIELD  = pick_basin_name(basins)  # "BNAME" in your case
BASIN_HYBAS_FIELD = pick_hybas(basins)
SUB_HYBAS_FIELD   = pick_hybas(subs)

if BASIN_NAME_FIELD is None:
    BASIN_NAME_FIELD = "BASIN_NAME"
    print(f"ℹ️ No basin name field found — creating '{BASIN_NAME_FIELD}' from {BASIN_HYBAS_FIELD}")
    basins[BASIN_NAME_FIELD] = "Basin_" + basins[BASIN_HYBAS_FIELD].astype(str)

print(f"🔎 Basin name field: {BASIN_NAME_FIELD}")
print(f"🔎 Basin HYBAS:      {BASIN_HYBAS_FIELD}")
print(f"🔎 Sub HYBAS:        {SUB_HYBAS_FIELD}")

# --- Preserve sub HYBAS and avoid name collisions ---
subs_for_join = subs.rename(columns={SUB_HYBAS_FIELD: "SUB_HYBAS_ID"})
basin_for_join = basins[[BASIN_NAME_FIELD, "geometry"]].rename(
    columns={BASIN_NAME_FIELD: "BASIN_NAME_JOIN"}
)

# Spatial join: within → fallback intersects
subs_tagged = gpd.sjoin(subs_for_join, basin_for_join, how="left", predicate="within").drop(columns=["index_right"])
unmatched = int(subs_tagged["BASIN_NAME_JOIN"].isna().sum())
print(f"🧩 Join 'within': unmatched sub-basins = {unmatched} / {len(subs_tagged)}")
if unmatched > 0:
    subs_try = gpd.sjoin(subs_for_join, basin_for_join, how="left", predicate="intersects").drop(columns=["index_right"])
    unmatched2 = int(subs_try["BASIN_NAME_JOIN"].isna().sum())
    if unmatched2 < unmatched:
        print(f"↪️ Switched to 'intersects' — unmatched now {unmatched2}")
        subs_tagged = subs_try

# --- Build BASIN_HYBAS_ID = <BASIN NAME (original case)> + " " + <SUB HYBAS ID with leading zeros preserved> ---
def normalize_id_str(x):
    s = str(x).strip()
    # remove trailing .0 if present (integers stored as float)
    if re.fullmatch(r"\d+\.0", s):
        return s[:-2]
    # if scientific notation or float-like but integral value, print as int
    try:
        f = float(s)
        if abs(f - round(f)) < 1e-9:
            return str(int(round(f)))
    except Exception:
        pass
    return s

raw_ids = subs_tagged["SUB_HYBAS_ID"].apply(normalize_id_str)

# auto-detect typical digit width to preserve leading zeros if any appear
digit_lengths = [len(re.sub(r"\D", "", v)) for v in raw_ids if isinstance(v, str) and re.search(r"\d", v)]
width = max(digit_lengths) if digit_lengths else None

def pad_like(v):
    if width is None: return v
    s = str(v)
    digits = re.sub(r"\D", "", s)
    if digits and len(digits) < width:
        padded = digits.zfill(width)
        return re.sub(r"\d+", lambda m: padded, s, count=1)
    return s

subs_tagged["SUB_HYBAS_ID_STR"] = raw_ids.apply(pad_like)

subs_tagged["BASIN_NAME"] = subs_tagged["BASIN_NAME_JOIN"]
subs_tagged["BASIN_HYBAS_ID"] = (
    subs_tagged["BASIN_NAME"].astype(str).str.strip()  # <-- keep original casing
    + " "
    + subs_tagged["SUB_HYBAS_ID_STR"]
)
subs_tagged["BASIN HYBAS ID"] = subs_tagged["BASIN_HYBAS_ID"]

# Save
OUT_GPKG = "/content/sub_basins_with_basin.gpkg"
OUT_LAYER = "sub_basins_result"
OUT_CSV  = "/content/sub_basins_with_basin_attributes.csv"
subs_tagged.to_file(OUT_GPKG, layer=OUT_LAYER, driver="GPKG")
subs_tagged.drop(columns="geometry").to_csv(OUT_CSV, index=False)

print("\n✅ Done!")
print(f"GeoPackage: {OUT_GPKG}  (layer: {OUT_LAYER})")
print(f"CSV:        {OUT_CSV}")
print(subs_tagged[["BASIN_NAME","SUB_HYBAS_ID_STR","BASIN_HYBAS_ID"]].head())

# Download
try:
    files.download(OUT_GPKG); files.download(OUT_CSV)
except Exception:
    print("If auto-download is blocked, use the Files pane (left) to download.")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
📤 Upload your TWO GeoPackages (basins & sub-basins)…


Saving Kenbasin_hydroshed_storage.gpkg to Kenbasin_hydroshed_storage (7).gpkg
Saving Kensubbasin_hydroshed_storage.gpkg to Kensubbasin_hydroshed_storage (7).gpkg
Uploaded: ['/content/Kenbasin_hydroshed_storage (7).gpkg', '/content/Kensubbasin_hydroshed_storage (7).gpkg']
🗂️ Basins file:     /content/Kenbasin_hydroshed_storage (7).gpkg
🗂️ Sub-basins file: /content/Kensubbasin_hydroshed_storage (7).gpkg
📚 Basin layer: Kenbasin_hydroshed_storage
📚 Sub layer:   Kensubbasin_hydroshed_storage
✅ Basins: 6 features, CRS=EPSG:4326
✅ Subs:   30 features, CRS=EPSG:4326
• Geographic CRS → reprojecting to EPSG:21037
• Geographic CRS → reprojecting to EPSG:21037
🔎 Basin name field: BNAME
🔎 Basin HYBAS:      HYBAS_ID
🔎 Sub HYBAS:        HYBAS_ID
🧩 Join 'within': unmatched sub-basins = 28 / 30
↪️ Switched to 'intersects' — unmatched now 0

✅ Done!
GeoPackage: /content/sub_basins_with_basin.gpkg  (layer: sub_basins_result)
CSV:        /content/sub_basins_with_basin_attributes.csv
    BASIN_NAME SUB_HYB

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>